# CB Recommender -- Build document use info for non-cold users 

### V3 - June 26th

This notebook continues the work in "Recsys-Snipet-CB-Topics-Analysis-V3" and creates a CB recommender solution based on the LDA previously created.


### Load interactions and transform to help latter calculations

In [29]:
## Load interactions

import numpy as np
import pandas as pd
import time

### DRAFT VERSION --- We need to agree on source files
def apiRecsysLoad12345Interactions():
    interactionsData=pd.read_pickle('datasets/interactions123DataAll.pkl')
    interactionsData=interactionsData.append(pd.read_pickle('datasets/interactions4Data.pkl'),ignore_index=False,verify_integrity=True)
    interactionsData=interactionsData.append(pd.read_pickle('datasets/interactions5Data.pkl'),ignore_index=False,verify_integrity=True)
    #interactionsData=interactionsData.append(pd.read_pickle('datasets/interactions0Data-lastweek.pkl'),ignore_index=False,verify_integrity=True)
    return interactionsData

#start timer
t_start = time.time()

interactionsData=apiRecsysLoad12345Interactions()

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

# Show structure
print interactionsData.shape

interactionsData.head(5)

Time invested 2.92199993134 s
(8274901, 5)


,user_id,item_id,interaction_type,created_at,created_at_date
0,2082156,80,1,1484299172,2017-01-13 09:19:32
1,1934123,140,1,1486388563,2017-02-06 13:42:43
2,1320213,240,1,1479409825,2016-11-17 19:10:25
3,297303,310,1,1484817366,2017-01-19 09:16:06
4,1635596,310,1,1486370081,2017-02-06 08:34:41


In [30]:
interactionsData.groupby(['interaction_type']).size()

interaction_type
1    6867579
2     281672
3     117843
4     906836
5     100971
dtype: int64

### Load items

In [31]:
## Load items
import numpy as np
import pandas as pd
import time

#start timer
t_start = time.time()

itemData=pd.read_table('datasets/items.csv',header=0,dtype=object,nrows=10000000)
itemData.columns = ['id','title','career_level','discipline_id','industry_id','country','is_payed','region',\
                    'latitude','longitude','employment','tags','created_at']

itemData['id']=itemData['id'].astype(np.int32)
itemData['title']=itemData['title'].apply(lambda x: [] if pd.isnull(x) else [np.int32(i) for i in x.split(',')])
itemData['career_level']=itemData['career_level'].astype(np.int8)
itemData['discipline_id']=itemData['discipline_id'].astype(np.int8)
itemData['industry_id']=itemData['industry_id'].astype(np.int8)
itemData['is_payed']=itemData['is_payed'].astype(np.int8)
itemData['region']=itemData['region'].astype(np.int8)
itemData['employment']=itemData['employment'].astype(np.int8)                             
itemData['tags']=itemData['tags'].apply(lambda x: [] if pd.isnull(x) else [np.int32(i) for i in x.split(',')])

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

# Show structure
print itemData.shape

Time invested 30.8639998436 s
(1306054, 13)


### Load users

In [32]:
import numpy as np
import pandas as pd
import time

#start timer
t_start = time.time()

userData=pd.read_table('datasets/users.csv',header=0,dtype=object)
userData.columns = ['id','jobroles','career_level','discipline_id','industry_id','country','region',\
                    'experience_n_entries_class','experience_years_experience','experience_years_in_current',\
                    'edu_degree','edu_fieldofstudies','wtcj','premium']

userData['id']=userData['id'].astype(np.int32)
userData['jobroles']=userData['jobroles'].apply(lambda x: [] if x=='0' else [np.int32(i) for i in x.split(',')])
userData['career_level']=userData['career_level'].astype(np.int8)
userData['discipline_id']=userData['discipline_id'].astype(np.int8)
userData['industry_id']=userData['industry_id'].astype(np.int8)
userData['region']=userData['region'].astype(np.int8)
userData['experience_n_entries_class']=userData['experience_n_entries_class'].astype(np.int8)
userData['experience_years_experience']=userData['experience_years_experience'].astype(np.int8)
userData['experience_years_in_current']=userData['experience_years_in_current'].astype(np.int8)
userData['edu_degree']=userData['edu_degree'].astype(np.int8)
userData['edu_fieldofstudies']=userData['edu_fieldofstudies'].apply(lambda x: [] if pd.isnull(x) else [np.int32(i) for i in x.split(',')])
userData['wtcj']=userData['wtcj'].astype(np.int8)
userData['premium']=userData['premium'].astype(np.int8)

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

# Show structure
print userData.shape

Time invested 18.611000061 s
(1497020, 14)


### Load target user/items

In [33]:
## LOAD TARGET ITEMS ##
#######################

%matplotlib inline
import numpy as np
import pandas as pd
import time
import operator

## Load the original targetItems.csv file or equivalent formatted one
## Return the dataframe appropiately adapted
## Note: original file does not have header --> user header=None
def apiRecsysLoadTargetItems(path='datasets/targetItems.csv'):
    targetItems=pd.read_table(path,header=None,dtype=np.int32)
    targetItems.columns=['item_id']
    return targetItems

#start timer
t_start = time.time()

targetItems=apiRecsysLoadTargetItems()

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

# Show structure
print targetItems.shape

## LOAD TARGET USERS ##
#######################

## Load the original targetUsers.csv file or equivalent formatted one
## Return the dataframe appropiately adapted
## Note: this original file does have header --> user header=0
def apiRecsysLoadTargetUsers(path='datasets/targetUsers.csv'):
    targetUsers=pd.read_table(path,header=0,dtype=np.int32)
    targetUsers.columns=['user_id']
    return targetUsers

#start timer
t_start = time.time()

targetUsers=apiRecsysLoadTargetUsers()

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

# Show structure
print targetUsers.shape

Time invested 0.0150001049042 s
(46559, 1)
Time invested 0.0319998264313 s
(74840, 1)


### Reduced interactions to target-user ones and do the pivot aggregation

In [34]:
userItemInt=interactionsData.groupby(['user_id','item_id','interaction_type']).size().unstack(fill_value=0.0)
userItemInt.columns=(['1','2','3','4','5'])
#userItemInt['0']=userItemInt['0'].astype(np.int16)
userItemInt['1']=userItemInt['1'].astype(np.int16)
userItemInt['2']=userItemInt['2'].astype(np.int16)
userItemInt['3']=userItemInt['3'].astype(np.int16)
userItemInt['4']=userItemInt['4'].astype(np.int16)
userItemInt['5']=userItemInt['5'].astype(np.int16)
userItemInt=userItemInt.reset_index()

In [35]:
print userItemInt.shape
userItemInt.head()

(5982813, 7)


,user_id,item_id,1,2,3,4,5
0,9,115748,1,0,0,0,0
1,9,536384,1,0,0,0,0
2,9,633504,1,0,0,0,0
3,9,720151,1,0,0,0,0
4,9,811877,1,0,0,0,0


In [36]:
targetUserItemInt=userItemInt.join(targetUsers.set_index(['user_id']),on='user_id',how='inner')
print targetUserItemInt.shape
targetUserItemInt.head()

(591698, 7)


,user_id,item_id,1,2,3,4,5
0,9,115748,1,0,0,0,0
1,9,536384,1,0,0,0,0
2,9,633504,1,0,0,0,0
3,9,720151,1,0,0,0,0
4,9,811877,1,0,0,0,0


In [37]:
targetUserItemInt=targetUserItemInt.join(userData.set_index(['id'])['premium'],on='user_id',how='left')
print targetUserItemInt.shape
targetUserItemInt.head()

(591698, 8)


,user_id,item_id,1,2,3,4,5,premium
0,9,115748,1,0,0,0,0,0
1,9,536384,1,0,0,0,0,0
2,9,633504,1,0,0,0,0,0
3,9,720151,1,0,0,0,0,0
4,9,811877,1,0,0,0,0,0


### Calculate the associated benefit of each user-item pair

In [38]:
targetUserItemInt['label']=[((1.0 if i1>0 else 0.0)+(5.0 if (i2>0 or i3>0) else 0.0)+(20.0 if i5>0 else 0.0)+\
                             (-10.0 if i1==0 and i2==0 and i3==0 and i4>0 and i5==0 else 0.0)
                            )*(2.0 if pr==1 else 1.0)
                            for i1,i2,i3,i4,i5,pr in zip(targetUserItemInt['1'],targetUserItemInt['2'],targetUserItemInt['3'],\
                                                      targetUserItemInt['4'],targetUserItemInt['5'],targetUserItemInt['premium'])]


In [39]:
print targetUserItemInt.shape
targetUserItemInt.head()

(591698, 9)


,user_id,item_id,1,2,3,4,5,premium,label
0,9,115748,1,0,0,0,0,0,1.0
1,9,536384,1,0,0,0,0,0,1.0
2,9,633504,1,0,0,0,0,0,1.0
3,9,720151,1,0,0,0,0,0,1.0
4,9,811877,1,0,0,0,0,0,1.0


### Build and join user rank (on all its overall positive interactions)

In [40]:
targetUsersRank=targetUserItemInt.groupby(['user_id'])['label'].sum().sort_values(ascending=False).to_frame()
print targetUsersRank.shape
targetUsersRank.head()

(51608, 1)


,label
user_id,
170003,2028.0
2292096,1584.0
2173340,1466.0
103997,1452.0
1073147,1100.0


In [41]:
## calculate norm_rank
targetUsersRank['norm_rank']=(targetUsersRank['label']-targetUsersRank['label'].min())/(targetUsersRank['label'].max()-targetUsersRank['label'].min())
print targetUsersRank.shape
targetUsersRank.head()

(51608, 2)


,label,norm_rank
user_id,,
170003,2028.0,1.000000
2292096,1584.0,0.968304
2173340,1466.0,0.959880
103997,1452.0,0.958881
1073147,1100.0,0.933752


In [42]:
targetUserItemInt=targetUserItemInt.join(targetUsersRank['norm_rank'],on='user_id')
print targetUserItemInt.shape
targetUserItemInt.head()

(591698, 10)


,user_id,item_id,1,2,3,4,5,premium,label,norm_rank
0,9,115748,1,0,0,0,0,0,1.0,0.855654
1,9,536384,1,0,0,0,0,0,1.0,0.855654
2,9,633504,1,0,0,0,0,0,1.0,0.855654
3,9,720151,1,0,0,0,0,0,1.0,0.855654
4,9,811877,1,0,0,0,0,0,1.0,0.855654


In [43]:
max_neg=max(targetUsersRank[targetUsersRank['label']<0]['norm_rank'])
min_neg=min(targetUsersRank[targetUsersRank['label']<0]['norm_rank'])
max_very_neg=max(targetUsersRank[targetUsersRank['label']<-30]['norm_rank'])
min_very_neg=min(targetUsersRank[targetUsersRank['label']<-30]['norm_rank'])
max_pos=max(targetUsersRank[targetUsersRank['label']>=0]['norm_rank'])
min_pos=min(targetUsersRank[targetUsersRank['label']>=0]['norm_rank'])
print max_neg,min_neg,max_very_neg,min_very_neg,max_pos,min_pos

0.855154197601 0.0 0.853012564249 0.0 1.0 0.85522558538


In [44]:
### Define negative threshold for latter use
NEGATIVE_THRESHOLD=0.855154197601
VERY_NEGATIVE_THRESHOLD=0.853012564249  

### Let me find where begins the "dangerous" users

In [45]:
print len(targetUsersRank['norm_rank'])
print len(targetUsersRank[targetUsersRank['norm_rank']<NEGATIVE_THRESHOLD])
print len(targetUsersRank[targetUsersRank['norm_rank']<VERY_NEGATIVE_THRESHOLD])

51608
6762
4861


### We can add the more recent positive interaction timestamp (and reduced only to positive interactions)

In [46]:
goodTargetUserItemInt=targetUserItemInt[targetUserItemInt['label']>0.0]
print goodTargetUserItemInt.shape

(458139, 10)


In [47]:
intForMaxTS=interactionsData[(interactionsData['interaction_type']>0) & (interactionsData['interaction_type']!=4)]
intForMaxTS=intForMaxTS.groupby(['user_id','item_id'])['created_at'].max()
intForMaxTS=intForMaxTS.to_frame()

In [48]:
goodTargetUserItemInt=goodTargetUserItemInt.join(intForMaxTS,on=['user_id','item_id'],how='left')
print goodTargetUserItemInt.shape
goodTargetUserItemInt.head()

(458139, 11)


,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,created_at
0,9,115748,1,0,0,0,0,0,1.0,0.855654,1485511373
1,9,536384,1,0,0,0,0,0,1.0,0.855654,1485511442
2,9,633504,1,0,0,0,0,0,1.0,0.855654,1484656586
3,9,720151,1,0,0,0,0,0,1.0,0.855654,1485511400
4,9,811877,1,0,0,0,0,0,1.0,0.855654,1485511337


### We can also add a indicator of the most recent activity of the user

In [49]:
## Add "recent" activity score (very basic)
intForMaxUserTS=interactionsData[(interactionsData['interaction_type']>0)]
intForMaxUserTS=intForMaxUserTS.groupby(['user_id'])['created_at'].max()
intForMaxUserTS=intForMaxUserTS.to_frame()
from datetime import date
intForMaxUserTS['recent']=[(7*date.fromtimestamp(i).isocalendar()[1]-7+date.fromtimestamp(i).isocalendar()[2]\
                            if date.fromtimestamp(i).isocalendar()[1]<=6 else 0) for i in intForMaxUserTS['created_at']]

## Add rankings
goodTargetUserItemInt=goodTargetUserItemInt.join(intForMaxUserTS['recent'],on='user_id',how='left')

print goodTargetUserItemInt.shape
goodTargetUserItemInt.head()

(458139, 12)


,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,created_at,recent
0,9,115748,1,0,0,0,0,0,1.0,0.855654,1485511373,26
1,9,536384,1,0,0,0,0,0,1.0,0.855654,1485511442,26
2,9,633504,1,0,0,0,0,0,1.0,0.855654,1484656586,26
3,9,720151,1,0,0,0,0,0,1.0,0.855654,1485511400,26
4,9,811877,1,0,0,0,0,0,1.0,0.855654,1485511337,26


### Let's add now the keywords involved in these items

In [50]:
goodTargetUserItemInt=goodTargetUserItemInt.join(itemData.set_index(['id'])[['title','discipline_id','industry_id','career_level','country','region','tags']],on='item_id',how='left')
print goodTargetUserItemInt.shape

(458139, 19)


In [51]:
goodTargetUserItemInt.head()

,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,created_at,recent,title,discipline_id,industry_id,career_level,country,region,tags
0,9,115748,1,0,0,0,0,0,1.0,0.855654,1485511373,26,[2337219],20,12,3,non_dach,0,"[22599, 2337219, 4491346, 3692743, 46526, 2585..."
1,9,536384,1,0,0,0,0,0,1.0,0.855654,1485511442,26,"[1562159, 2337219]",20,9,3,de,3,"[2705743, 2337219, 2486448, 1562159, 107434, 1..."
2,9,633504,1,0,0,0,0,0,1.0,0.855654,1484656586,26,[],20,16,3,de,3,"[986208, 2652564, 3690613, 3128825, 51487, 233..."
3,9,720151,1,0,0,0,0,0,1.0,0.855654,1485511400,26,[],20,16,3,de,3,"[3671787, 1739351, 2337219, 107434, 2486448]"
4,9,811877,1,0,0,0,0,0,1.0,0.855654,1485511337,26,"[645566, 2337219]",20,12,3,de,3,"[2447971, 2337219, 640078, 4509781, 3356528, 3..."


In [52]:
from datetime import date

goodTargetUserItemInt['weeknum']=[date.fromtimestamp(i).isocalendar()[1] for i in goodTargetUserItemInt['created_at']]
print sorted(goodTargetUserItemInt['weeknum'].unique())

[1, 2, 3, 4, 5, 6, 44, 45, 46, 50, 52]


In [53]:
### Looking at the range of weeknum ajust manually past year weeknum
goodTargetUserItemInt['weeknum_rank']=[(i+(52-44))%52 for i in goodTargetUserItemInt['weeknum']]
print sorted(goodTargetUserItemInt['weeknum_rank'].unique())

[0, 1, 2, 6, 8, 9, 10, 11, 12, 13, 14]


In [54]:
goodTargetUserItemInt.head()

,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,...,recent,title,discipline_id,industry_id,career_level,country,region,tags,weeknum,weeknum_rank
0,9,115748,1,0,0,0,0,0,1.0,0.855654,...,26,[2337219],20,12,3,non_dach,0,"[22599, 2337219, 4491346, 3692743, 46526, 2585...",4,12
1,9,536384,1,0,0,0,0,0,1.0,0.855654,...,26,"[1562159, 2337219]",20,9,3,de,3,"[2705743, 2337219, 2486448, 1562159, 107434, 1...",4,12
2,9,633504,1,0,0,0,0,0,1.0,0.855654,...,26,[],20,16,3,de,3,"[986208, 2652564, 3690613, 3128825, 51487, 233...",3,11
3,9,720151,1,0,0,0,0,0,1.0,0.855654,...,26,[],20,16,3,de,3,"[3671787, 1739351, 2337219, 107434, 2486448]",4,12
4,9,811877,1,0,0,0,0,0,1.0,0.855654,...,26,"[645566, 2337219]",20,12,3,de,3,"[2447971, 2337219, 640078, 4509781, 3356528, 3...",4,12


In [55]:
finalGoodTargetUserItemInt=goodTargetUserItemInt

### Save copy of this final good/positive users-item interactions (for target user) with:

 - interactions type counts on this user-item pair
 - associated overall label score for this user-item pair
 - weeknum (and weeknum_rank) of last actitivy in this user-item pair
 - norm_rank of the users involved in this set related to the total sum of benefit (label) the bring
 - keywords and feature-keyword-use field associated to the item part of the user-item pair

We will use all of this to get most relevant documents (from targets) using our LDA and a similirity matrix (to build) weighted by label/interaction_counts plus weeknum_rank plus norm_rank.
We will need to use several candidates of weighted rules.

With that we will build a list of [target_user,recommended_target_item,weigthed_similarity_score].
Atferwards we need to use that to get for each item the first 100 more relevant users (or less if we use a threeshold)


NOTE: as 99,9% of target items are cold, makes no sense to check if the relevant documents retrieved were already interacted by the user.

In [56]:
finalGoodTargetUserItemInt.to_pickle('datasets/goodNonColdTargetUserInt.pkl')

### Build final combined weights for recommedation -- We may try several options

NOTE: Probaby good idea to add premium to increase the score --> That will make the premium user to be recommeded more times

In [36]:
### Optional reload of good non-cold target user interactions
import pandas as pd
import numpy as np
import time

finalGoodTargetUserItemInt=pd.read_pickle('datasets/goodNonColdTargetUserInt.pkl')

In [58]:
print finalGoodTargetUserItemInt.shape
print finalGoodTargetUserItemInt.columns
finalGoodTargetUserItemInt.head()

(458139, 21)
Index([u'user_id', u'item_id', u'1', u'2', u'3', u'4', u'5', u'premium',
       u'label', u'norm_rank', u'created_at', u'recent', u'title',
       u'discipline_id', u'industry_id', u'career_level', u'country',
       u'region', u'tags', u'weeknum', u'weeknum_rank'],
      dtype='object')


,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,...,recent,title,discipline_id,industry_id,career_level,country,region,tags,weeknum,weeknum_rank
0,9,115748,1,0,0,0,0,0,1.0,0.855654,...,26,[2337219],20,12,3,non_dach,0,"[22599, 2337219, 4491346, 3692743, 46526, 2585...",4,12
1,9,536384,1,0,0,0,0,0,1.0,0.855654,...,26,"[1562159, 2337219]",20,9,3,de,3,"[2705743, 2337219, 2486448, 1562159, 107434, 1...",4,12
2,9,633504,1,0,0,0,0,0,1.0,0.855654,...,26,[],20,16,3,de,3,"[986208, 2652564, 3690613, 3128825, 51487, 233...",3,11
3,9,720151,1,0,0,0,0,0,1.0,0.855654,...,26,[],20,16,3,de,3,"[3671787, 1739351, 2337219, 107434, 2486448]",4,12
4,9,811877,1,0,0,0,0,0,1.0,0.855654,...,26,"[645566, 2337219]",20,12,3,de,3,"[2447971, 2337219, 640078, 4509781, 3356528, 3...",4,12


In [59]:
## New version (adjusted + premium)
def apiRecsysDocWeightCorr(row):
    weight=1.0
    if (row['1']>0):
        weight=weight+(np.log2(row['1'])/200.0)
    if (row['2']>0 & row['3']>0):
        weight=weight+0.05
    if (row['2']>0 & row['3']==0):
        weight=weight+0.04 
    if (row['2']==0 & row['3']>0):
        weight=weight+0.04
    if (row['5']>0):
        weight=weight+0.12
    if (row['premium']):
        weight=weight*1.45 
    ## Apply weeknum
    weight=weight*(1.0+np.log2(1.0+row['weeknum_rank'])/200.0)
    ## Apply user last activity
    weight=weight*(1.0+np.log2(1.0+60.0*row['recent'])/500.0)
    if row['recent']>=20:
        weight=weight*1.01
    if row['recent']>=30:
        weight=weight*1.015        
    ## Apply user_rank
    if row['norm_rank']<=VERY_NEGATIVE_THRESHOLD:
        weight=weight*0.25
    else:
        if row['norm_rank']<=NEGATIVE_THRESHOLD:
            weight=weight*0.75
        else:
            weight=weight*(1.0+np.log2(1.0+25*(row['norm_rank']-NEGATIVE_THRESHOLD))/50.0) 
    return weight

#start timer
t_start = time.time()                               

finalGoodTargetUserItemInt['score_weight']=finalGoodTargetUserItemInt.apply(lambda row: apiRecsysDocWeightCorr(row),axis=1)
print finalGoodTargetUserItemInt.shape
finalGoodTargetUserItemInt.head()

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

(458139, 22)
Time invested 75.0429999828 s


In [60]:
print finalGoodTargetUserItemInt.shape
finalGoodTargetUserItemInt.head()

(458139, 22)


,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,...,title,discipline_id,industry_id,career_level,country,region,tags,weeknum,weeknum_rank,score_weight
0,9,115748,1,0,0,0,0,0,1.0,0.855654,...,[2337219],20,12,3,non_dach,0,"[22599, 2337219, 4491346, 3692743, 46526, 2585...",4,12,1.050889
1,9,536384,1,0,0,0,0,0,1.0,0.855654,...,"[1562159, 2337219]",20,9,3,de,3,"[2705743, 2337219, 2486448, 1562159, 107434, 1...",4,12,1.050889
2,9,633504,1,0,0,0,0,0,1.0,0.855654,...,[],20,16,3,de,3,"[986208, 2652564, 3690613, 3128825, 51487, 233...",3,11,1.050293
3,9,720151,1,0,0,0,0,0,1.0,0.855654,...,[],20,16,3,de,3,"[3671787, 1739351, 2337219, 107434, 2486448]",4,12,1.050889
4,9,811877,1,0,0,0,0,0,1.0,0.855654,...,"[645566, 2337219]",20,12,3,de,3,"[2447971, 2337219, 640078, 4509781, 3356528, 3...",4,12,1.050889


### Let's reload our item LDA, build the corpus (only on target items) and the similarity matrix

#### Build the target items set 

In [61]:
targetItemData=itemData.join(targetItems.set_index(['item_id']),on='id',how='inner').reset_index(drop=True)
print targetItemData.shape
targetItemData.head()

(46559, 13)


,id,title,career_level,discipline_id,industry_id,country,is_payed,region,latitude,longitude,employment,tags,created_at
0,260,"[1217406, 2070620, 2773698]",4,8,16,de,0,10,51.2,7.1,1,"[1827822, 2773698, 1217406, 2785572, 1786537]",1480546800
1,300,"[3315162, 1789447, 2454373, 1482448]",3,15,16,de,0,5,52.7,8.3,1,"[2815721, 3672742, 3315162, 1482448, 1789447, ...",1480633200
2,660,"[1482448, 4311326]",3,15,6,de,0,2,48.2,11.6,1,"[186705, 1482448, 3885883, 4311326, 3505274]",1482102000
3,720,[2842161],3,3,7,de,0,1,49.4,8.1,1,"[1443992, 4054733, 2842161, 1041753, 3553130, ...",1480460400
4,820,[1337363],3,3,7,de,0,13,50.8,12.9,1,"[4059762, 3671787, 4043733, 4208896, 2432628, ...",1473372000


#### Rebuild the corpus used for items-lda (for onñy for target items)

In [62]:
def apyRecsysItemToBow(item):
    extra=[]
    if (item.discipline_id>0):
        extra=extra+["disc"+str(item.discipline_id)]
    if (item.industry_id>0):
        extra=extra+["indus"+str(item.industry_id)]
    if (item.career_level>0):
        extra=extra+["career"+str(item.career_level)]
    extra=extra+[item.country]
    if (item.region>0):
        extra=extra+["reg"+str(item.region)]
    bow=",".join(map(str,extra+item['title']+item['tags']))
    return bow

#start timer
t_start = time.time()                               

targetDocs=targetItemData.apply(lambda row: apyRecsysItemToBow(row),axis=1).apply(lambda row: row.split(","))

#end timer
t_end=time.time()
print "Time invested (create dictionary)"+str((t_end-t_start))+" s"

print targetDocs.shape
targetDocs.head(10)

Time invested (create dictionary)7.48900008202 s
(46559L,)


0    [disc8, indus16, career4, de, reg10, 1217406, ...
1    [disc15, indus16, career3, de, reg5, 3315162, ...
2    [disc15, indus6, career3, de, reg2, 1482448, 4...
3    [disc3, indus7, career3, de, reg1, 2842161, 14...
4    [disc3, indus7, career3, de, reg13, 1337363, 4...
5    [disc13, indus9, career3, de, reg3, 2398283, 2...
6    [disc15, indus7, career3, de, reg3, 1168089, 2...
7    [disc13, indus9, career3, de, reg9, 1474572, 8...
8    [disc8, indus4, career3, de, reg10, 2942918, 1...
9    [disc18, indus2, career3, de, reg4, 2945601, 4...
dtype: object

#### Reload dictionary, LDA model and build memory similiraty index

In [63]:
import gensim
from gensim import corpora, models, similarities

# Reload dictionary associate to our LDA
dictionary = corpora.Dictionary.load('lda/recsys_dict_300topic.pkl')
print dictionary

#corpus here is ours "docs" list 
#lda_model is our latent space model
# we can reload from disk if need be
lda_model=gensim.models.ldamulticore.LdaMulticore.load("lda/ldamodel_300topic.pkl")

# We need to rebuild doc_term_matrix == "corpus"
doc_term_matrix=[dictionary.doc2bow(doc) for doc in targetDocs] 

# we can create a similarity matrix --> we can use our latent space directly or use it on other corpus
##index=similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

# First we need to transform our "corpus" to our LDA space model
#start timer
t_start = time.time()                          

lda_corpus=lda_model[doc_term_matrix]

# end timer
t_end=time.time()
print "Time invested (lda_corpus)"+str((t_end-t_start))+" s"

# check topics a N features
n_features=len(lda_model.show_topics(-1))
print n_features

#start timer
t_start=time.time()

## If very big corpus let's go for disk matrix
#index_disk=similarities.Similarity(output_prefix='lda/index/ldaindex',num_features=50,corpus=lda_corpus,num_best=None, chunksize=256, shardsize=32768)
#index_disk.save("lda/index/ldaindex.pkl")

# In memory (may not resist big numbers)
index_mem=similarities.MatrixSimilarity(lda_corpus) # transform corpus to LDA space and index it

#end timer
t_end=time.time()
print "Time invested (index_mem)"+str((t_end-t_start))+" s"

#index.save('lda/index/deerwester.index')
#index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

C:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Dictionary(98470 unique tokens: [u'2668995', u'2553225', u'2983263', u'839790', u'2970301']...)
Time invested (lda_corpus)0.0 s
300
Time invested (index_mem)439.436000109 s


#### Do some test to verify makes sense all built

In [64]:
targetDocs[0]

['disc8',
 'indus16',
 'career4',
 'de',
 'reg10',
 '1217406',
 '2070620',
 '2773698',
 '1827822',
 '2773698',
 '1217406',
 '2785572',
 '1786537']

In [65]:
## Do a similarity query

## We out most words from first targetDocs and do the query
query="disc8 indus16 career4 de reg10 1217406 2070620 2773698 1827822"
vec_bow=dictionary.doc2bow(query.split())
vec_lda=lda_model[vec_bow] # convert the query to LDA space
#print(vec_lda)

index_mem.num_best=10
sims=index_mem[vec_lda] # perform a similarity query against the corpus
#print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples
#sims=sorted(enumerate(sims), key=lambda item: -item[1])[0:10]
print(sims) # print sorted (document number, similarity score) 2-tuples

## Check the 0 document (first) appears as top document

[(0, 0.99160033464431763), (12950, 0.94719409942626953), (20908, 0.94366240501403809), (33964, 0.94330930709838867), (38721, 0.94183808565139771), (27312, 0.94106215238571167), (32734, 0.9391663670539856), (11034, 0.93600475788116455), (40083, 0.9335024356842041), (5853, 0.93241715431213379)]


### We can now generate initial final list of non-cold-users best related items

we list top 5 interactions for every user and for each of them get ten related docs that we will weight

In [66]:
TOP_N=20
TOP_N=0 ### ALL

print finalGoodTargetUserItemInt.shape
if TOP_N>0:
    topCandidateInt=finalGoodTargetUserItemInt.sort_values(['user_id','score_weight'],ascending=[True,False]).groupby(['user_id']).head(TOP_N)
    print topCandidateInt.shape 
else:
    ## Alternative --> Not to filter
    topCandidateInt=finalGoodTargetUserItemInt
    print topCandidateInt.shape

(458139, 22)
(458139, 22)


In [70]:
topCandidateInt=topCandidateInt.reset_index(drop=True)
topCandidateInt.head()

,user_id,item_id,1,2,3,4,5,premium,label,norm_rank,...,title,discipline_id,industry_id,career_level,country,region,tags,weeknum,weeknum_rank,score_weight
0,9,115748,1,0,0,0,0,0,1.0,0.855654,...,[2337219],20,12,3,non_dach,0,"[22599, 2337219, 4491346, 3692743, 46526, 2585...",4,12,1.050889
1,9,536384,1,0,0,0,0,0,1.0,0.855654,...,"[1562159, 2337219]",20,9,3,de,3,"[2705743, 2337219, 2486448, 1562159, 107434, 1...",4,12,1.050889
2,9,633504,1,0,0,0,0,0,1.0,0.855654,...,[],20,16,3,de,3,"[986208, 2652564, 3690613, 3128825, 51487, 233...",3,11,1.050293
3,9,720151,1,0,0,0,0,0,1.0,0.855654,...,[],20,16,3,de,3,"[3671787, 1739351, 2337219, 107434, 2486448]",4,12,1.050889
4,9,811877,1,0,0,0,0,0,1.0,0.855654,...,"[645566, 2337219]",20,12,3,de,3,"[2447971, 2337219, 640078, 4509781, 3356528, 3...",4,12,1.050889


In [71]:
topCandidateInt['score_weight'].describe()

count    458139.000000
mean          1.084757
std           0.325419
min           0.251250
25%           1.040243
50%           1.067037
75%           1.092677
max           1.822725
Name: score_weight, dtype: float64

In [72]:
from operator import itemgetter

def getTopUserItemIntDocsOld(row,dictionary,model,index,funcBow,limit,TH):
    score_weight=row['score_weight']
    vecBow=dictionary.doc2bow(map(str,funcBow(row).split(",")))
    vecModel=model[vecBow]
    sims=index[vecModel] # perform a similarity query against the corpus
    result=[[x[0],x[1],row['user_id'],row['item_id'],x[1]*score_weight] for x in sorted(enumerate(sims),key=lambda item: -item[1])[0:limit] if x[1]>=TH]
    return result

## This version removes "limit" but expects index.num_best to be set
def getTopUserItemIntDocs(row,dictionary,model,index,funcBow,TH):
    score_weight=row['score_weight']
    vecBow=dictionary.doc2bow(map(str,funcBow(row).split(",")))
    vecModel=model[vecBow]
    sims=index[vecModel] # perform a similarity query against the corpus
    result=[[x[0],x[1],row['user_id'],row['item_id'],x[1]*score_weight] for x in sims if x[1]>=TH]
    return result

print topCandidateInt.shape

NUM_BEST_ITEMS=50
SIM_THRESHOLD=0.45

## Inits
#index_mem.num_best=None
index_mem.num_best=NUM_BEST_ITEMS

#start timer
t_start = time.time()                               

aux=topCandidateInt #[0:5001]
totalData=[]
iterData=[]
for ind in aux.index: #topCandidateInt.index:
    if (ind%2500==0):
        #start timer
        t_start_inner=time.time() 
    #rowData=getTopUserItemIntDocsOld(topCandidateInt.iloc[ind],dictionary,lda_model,index_mem,apyRecsysItemToBow,25,0.75)
    rowData=getTopUserItemIntDocs(topCandidateInt.iloc[ind],dictionary,lda_model,index_mem,apyRecsysItemToBow,SIM_THRESHOLD)
    iterData=iterData+rowData
    if ((ind+1)%2500==0):
        totalData=totalData+iterData
        iterData=[]
        #end timer
        t_end_inner=time.time()
        print "checkpoint at index",ind," - Time "+str((t_end_inner-t_start_inner))+" s"
if len(iterData)>0:
    totalData=totalData+iterData
               
topUserItemDocs=pd.DataFrame(totalData,columns=['item_index_corpus','score','user_id','source_item_id','score_weight'])       
        
#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"
        
print topUserItemDocs.shape
topUserItemDocs.head()

(458139, 22)
checkpoint at index 2499  - Time 54.5970001221 s
checkpoint at index 4999  - Time 55.146999836 s
checkpoint at index 7499  - Time 54.3950002193 s
checkpoint at index 9999  - Time 54.2319998741 s
checkpoint at index 12499  - Time 52.8150000572 s
checkpoint at index 14999  - Time 53.1210000515 s
checkpoint at index 17499  - Time 53.3839998245 s
checkpoint at index 19999  - Time 53.8970000744 s
checkpoint at index 22499  - Time 53.5929999352 s
checkpoint at index 24999  - Time 53.1000001431 s
checkpoint at index 27499  - Time 56.7079999447 s
checkpoint at index 29999  - Time 52.9839999676 s
checkpoint at index 32499  - Time 59.6890001297 s
checkpoint at index 34999  - Time 58.871999979 s
checkpoint at index 37499  - Time 58.4299998283 s
checkpoint at index 39999  - Time 57.4609999657 s
checkpoint at index 42499  - Time 57.7730000019 s
checkpoint at index 44999  - Time 62.739000082 s
checkpoint at index 47499  - Time 62.5910000801 s
checkpoint at index 49999  - Time 66.3529999

,item_index_corpus,score,user_id,source_item_id,score_weight
0,16176,0.912683,9,115748,0.959128
1,18660,0.891382,9,115748,0.936743
2,28538,0.876825,9,115748,0.921446
3,20425,0.873014,9,115748,0.917441
4,13560,0.871584,9,115748,0.915938


In [96]:
print topUserItemDocs.shape
print topUserItemDocs.columns
#topUserItemDocs.columns=['item_index_corpus','score','user_id','source_item_id','score_weight']
#print topUserItemDocs.shape
#print topUserItemDocs.columns
topUserItemDocs.head()

(18631751, 5)
Index([u'item_index_corpus', u'score', u'user_id', u'source_item_id',
       u'score_weight'],
      dtype='object')


,item_index_corpus,score,user_id,source_item_id,score_weight
0,16176,0.912535,9,115748,0.945783
1,18660,0.891203,9,115748,0.923674
2,28538,0.876615,9,115748,0.908554
3,20425,0.872812,9,115748,0.904612
4,13560,0.871415,9,115748,0.903165


#### Remenber similarity matrix retrieves the index-order in the corpus (0..N)...we need to join again to targetItems to reoover the target item id


In [97]:
print topUserItemDocs.shape
topUserItemDocs=topUserItemDocs.join(targetItemData['id'],on='item_index_corpus',how='left')
print topUserItemDocs.shape
topUserItemDocs.head()

(18631751, 5)
(18631751, 6)


,item_index_corpus,score,user_id,source_item_id,score_weight,id
0,16176,0.912535,9,115748,0.945783,1007373
1,18660,0.891203,9,115748,0.923674,2226083
2,28538,0.876615,9,115748,0.908554,193957
3,20425,0.872812,9,115748,0.904612,829735
4,13560,0.871415,9,115748,0.903165,2016182


In [98]:
topUserItemDocs.columns=['item_index_corpus','score','user_id','source_item_id','score_weight','item_id']
topUserItemDocs.head()

,item_index_corpus,score,user_id,source_item_id,score_weight,item_id
0,16176,0.912535,9,115748,0.945783,1007373
1,18660,0.891203,9,115748,0.923674,2226083
2,28538,0.876615,9,115748,0.908554,193957
3,20425,0.872812,9,115748,0.904612,829735
4,13560,0.871415,9,115748,0.903165,2016182


In [99]:
topUserItemDocs.to_pickle('datasets/topUserItemDocsCB_27Jun_vFULL300.pkl')

### Now we can answer the questions of given a item take best users (using the score_weight)

In [5]:
import pandas as pd
import numpy as np

topUserItemDocs=pd.read_pickle('datasets/topUserItemDocsCB_27Jun_vFULL300.pkl')

In [6]:
## First assess how many different targets items we're covering 
len(topUserItemDocs['item_id'].unique())

46442

In [7]:
## check how many different users
len(topUserItemDocs['user_id'].unique())

## Should match the target non-cold-users aprox.

50164

In [8]:
resultItemUsersLen=topUserItemDocs.groupby(['item_id']).size()

print min(resultItemUsersLen)
print max(resultItemUsersLen)
print np.median(resultItemUsersLen)
print np.mean(resultItemUsersLen)

1
14888
162.0
401.183217777


### Let's write recomendation file to test

In [26]:
## Check avg number of distinct user per item --> For full use we should approach 100?
##topUserItemDocs.groupby(['item_id'])['user_id'].zip()

## Here we tried several another tresholds
TH=0.45

results=topUserItemDocs[topUserItemDocs['score']>TH].groupby(['item_id']).apply(lambda g: [x[0] for x in sorted(zip(g['user_id'],g['score_weight']),key=lambda t: -t[1])[0:100]])
print results.shape
print results.head()

(46442L,)
item_id
103    [968548, 1121184, 240124, 169200, 590618, 7095...
113                                            [1235656]
260    [865895, 2109770, 865895, 878845, 311024, 6843...
289    [1480022, 1480022, 2253016, 2253016, 968984, 4...
300    [542043, 865895, 1741772, 686307, 1035963, 191...
dtype: object


In [27]:
results=results.to_frame().reset_index()
results.columns=['item_id','users']
results.head()

,item_id,users
0,103,"[968548, 1121184, 240124, 169200, 590618, 7095..."
1,113,[1235656]
2,260,"[865895, 2109770, 865895, 878845, 311024, 6843..."
3,289,"[1480022, 1480022, 2253016, 2253016, 968984, 4..."
4,300,"[542043, 865895, 1741772, 686307, 1035963, 191..."


In [28]:
stats=[len(x) for x in results['users']]
print min(stats)
print max(stats)
print np.median(stats)
print np.mean(stats)

1
100
100.0
79.2790362172


In [22]:
### Let's generate results file
with open('cb_solution_Jun27_055_vfull300.csv','w') as fp:
    for ind in results.index:
        item_id=str(results['item_id'][ind])+"\t"
        fp.write(item_id)
        users=",".join(map(str,results['users'][ind]))   
        fp.write(users)
        fp.write("\n")
        fp.flush()
fp.close()
        

### Extra ideas

In [ ]:
print len(topUserItemDocs['item_id'].unique())

In [392]:
np.mean(topUserItemDocs['score'])

0.8364359223892484